In [13]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torch.optim.lr_scheduler import StepLR

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 256
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()])

# dataset = MNIST('./data', transform=img_transform, download=True)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# load the training and test datasets
train_data = MNIST(root='./data', train=True,
                                   download=True, transform=img_transform)
test_data = MNIST(root='./data', train=False,
                                  download=True, transform=img_transform)

# dataset = MNIST('./data', transform=img_transform, download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [15]:
class Cons_lin_layer(nn.Module):
    def __init__(self, n_in, n_out, nonlin = 'tanh', p = 2):
        super(Cons_lin_layer,self).__init__()
        self.lin = nn.Linear(n_in,n_out)
        if nonlin == 'none':
            self.layer = self.lin
        else:
            if nonlin == 'tanh':
                self.nonlin = nn.Tanh()
            elif nonlin == 'relu':
                self.nonlin = nn.ReLU()
            elif nonlin == 'sigmoid':
                self.nonlin = nn.Sigmoid()
            self.layer = nn.Sequential(self.lin,self.nonlin)
       
        self.p = p
   
    def forward(self, x):
        energy = x.norm(p=self.p, dim=-1, keepdim=True)
        y = self.layer(x)
        y = F.normalize(y,p=self.p,dim=-1)*energy
        return y

In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.encoder = nn.Sequential(
            Cons_lin_layer(28*28,128, nonlin='relu'),
            Cons_lin_layer(128,64, nonlin='relu'),
            Cons_lin_layer(64,32, nonlin='relu'),
            Cons_lin_layer(32,10, nonlin='none'))
    def forward(self, x):
        x = self.encoder(x)
        output = F.log_softmax(x, dim=1)
        return output

In [27]:
class Net_simple(nn.Module):
    def __init__(self):
        super(Net_simple, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28,128),nn.ReLU(),
            nn.Linear(128,64),nn.ReLU(),
            nn.Linear(64,32),nn.ReLU(),
            nn.Linear(32,10))
    def forward(self, x):
        x = self.encoder(x)
        output = F.log_softmax(x, dim=1)
        return output

In [24]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        img, target = data.to(device), target.to(device)
        img = img.view(img.size(0), -1)
        optimizer.zero_grad()
        output = model(img)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            img, target = data.to(device), target.to(device)
            img = img.view(img.size(0), -1)
            output = model(img)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 100 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 5.688402
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.428929
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.281084
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.334946
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.210201

Test set: Average loss: 0.2642, Accuracy: 9198/10000 (91.98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.253367
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.211514
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.082486
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.086865
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.055200

Test set: Average loss: 0.1414, Accuracy: 9571/10000 (95.71%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.062514
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.112142
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.100238
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.070009
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.119058

Test set: Average loss: 0.0959, Accuracy: 9713/10000 (97.13%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.037492
Train Epoch: 4 [12800/60000 

Train Epoch: 27 [25600/60000 (43%)]	Loss: 0.009175
Train Epoch: 27 [38400/60000 (64%)]	Loss: 0.007740
Train Epoch: 27 [51200/60000 (85%)]	Loss: 0.010856

Test set: Average loss: 0.0715, Accuracy: 9808/10000 (98.08%)

Train Epoch: 28 [0/60000 (0%)]	Loss: 0.005035
Train Epoch: 28 [12800/60000 (21%)]	Loss: 0.017998
Train Epoch: 28 [25600/60000 (43%)]	Loss: 0.007172
Train Epoch: 28 [38400/60000 (64%)]	Loss: 0.007425
Train Epoch: 28 [51200/60000 (85%)]	Loss: 0.037387

Test set: Average loss: 0.0715, Accuracy: 9808/10000 (98.08%)

Train Epoch: 29 [0/60000 (0%)]	Loss: 0.005593
Train Epoch: 29 [12800/60000 (21%)]	Loss: 0.006564
Train Epoch: 29 [25600/60000 (43%)]	Loss: 0.008968
Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.014583
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.011685

Test set: Average loss: 0.0715, Accuracy: 9808/10000 (98.08%)

Train Epoch: 30 [0/60000 (0%)]	Loss: 0.013103
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.006800
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.007398
Train E

KeyboardInterrupt: 

In [28]:
model2 = Net_simple().to(device)
optimizer2 = optim.Adadelta(model2.parameters(), lr=1)

scheduler2 = StepLR(optimizer2, step_size=1, gamma=0.7)
for epoch in range(1, 100 + 1):
    train(model2, device, train_loader, optimizer2, epoch)
    test(model2, device, test_loader)
    scheduler2.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308327
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.118652
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.616027
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.395210
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.249319

Test set: Average loss: 0.2236, Accuracy: 9305/10000 (93.05%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.230193
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.209047
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.088444
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.200958
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.115025

Test set: Average loss: 0.1482, Accuracy: 9554/10000 (95.54%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.145381
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.188354
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.114507
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.146574
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.148268

Test set: Average loss: 0.1167, Accuracy: 9633/10000 (96.33%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.105057
Train Epoch: 4 [12800/60000 

Train Epoch: 27 [25600/60000 (43%)]	Loss: 0.025037
Train Epoch: 27 [38400/60000 (64%)]	Loss: 0.073724
Train Epoch: 27 [51200/60000 (85%)]	Loss: 0.058464

Test set: Average loss: 0.0722, Accuracy: 9774/10000 (97.74%)

Train Epoch: 28 [0/60000 (0%)]	Loss: 0.029611
Train Epoch: 28 [12800/60000 (21%)]	Loss: 0.033628
Train Epoch: 28 [25600/60000 (43%)]	Loss: 0.036297
Train Epoch: 28 [38400/60000 (64%)]	Loss: 0.020316
Train Epoch: 28 [51200/60000 (85%)]	Loss: 0.034646

Test set: Average loss: 0.0722, Accuracy: 9774/10000 (97.74%)

Train Epoch: 29 [0/60000 (0%)]	Loss: 0.075115
Train Epoch: 29 [12800/60000 (21%)]	Loss: 0.059412
Train Epoch: 29 [25600/60000 (43%)]	Loss: 0.081342
Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.047678
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.042734

Test set: Average loss: 0.0722, Accuracy: 9774/10000 (97.74%)

Train Epoch: 30 [0/60000 (0%)]	Loss: 0.032532
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.026808
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.046884
Train E

Train Epoch: 53 [38400/60000 (64%)]	Loss: 0.017851
Train Epoch: 53 [51200/60000 (85%)]	Loss: 0.041819

Test set: Average loss: 0.0722, Accuracy: 9774/10000 (97.74%)

Train Epoch: 54 [0/60000 (0%)]	Loss: 0.045109
Train Epoch: 54 [12800/60000 (21%)]	Loss: 0.035909
Train Epoch: 54 [25600/60000 (43%)]	Loss: 0.060211
Train Epoch: 54 [38400/60000 (64%)]	Loss: 0.054194
Train Epoch: 54 [51200/60000 (85%)]	Loss: 0.025424

Test set: Average loss: 0.0722, Accuracy: 9774/10000 (97.74%)

Train Epoch: 55 [0/60000 (0%)]	Loss: 0.023101
Train Epoch: 55 [12800/60000 (21%)]	Loss: 0.029366
Train Epoch: 55 [25600/60000 (43%)]	Loss: 0.049794
Train Epoch: 55 [38400/60000 (64%)]	Loss: 0.044426
Train Epoch: 55 [51200/60000 (85%)]	Loss: 0.068650

Test set: Average loss: 0.0722, Accuracy: 9774/10000 (97.74%)

Train Epoch: 56 [0/60000 (0%)]	Loss: 0.055358
Train Epoch: 56 [12800/60000 (21%)]	Loss: 0.039841
Train Epoch: 56 [25600/60000 (43%)]	Loss: 0.056943
Train Epoch: 56 [38400/60000 (64%)]	Loss: 0.068022
Train E

KeyboardInterrupt: 